In [1]:
import dask

if dask.__version__:
    print(f"Dask version {dask.__version__} is installed.")
else:
    print("Dask is not installed. You can install it using 'conda install dask' or 'pip install dask[complete]'.")

Dask version 2022.7.0 is installed.


In [2]:
import pandas as pd
import dask.dataframe as dd

from pathlib import Path
from sqlalchemy import create_engine, text


In [4]:
file_path = r"C:/Desktop/Analysis Projects/LOC Subset.csv"

# Specifying encoding as'ISO-8859-1' to read file
loc_df = dd.read_csv(file_path, encoding='ISO-8859-1')

In [7]:
#import csv file
file_path = r"C:/Desktop/Analysis Projects/LOC Subset.csv"

# Specifying encoding as'ISO-8859-1' to read file
loc_df = pd.read_csv(file_path, encoding='ISO-8859-1')

In [10]:
loc_df.head()


,wdl_id,title,collection,description,title_in_original_language,wdl_url,handle,Random,Type of Item,Institution,...,Singer,Binder,Originator,Typographer,Book producer,Narrator,Colorist,Arranger,Recipient,Correspondent
0,4315,Opening Proclamation from University Authoriti...,NaN,The University of Vienna was founded by Duke R...,NaN,https://www.wdl.org/en/item/4315,https://hdl.loc.gov/loc.wdl/wdl.4315,1.0,Books,Austrian National Library,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7787,History of the Revolution in New Spain,NaN,Fray Servando Teresa de Mier was born in Monte...,Historia de la Revolución de Nueva España,https://www.wdl.org/en/item/7787,https://hdl.loc.gov/loc.wdl/wdl.7787,1.0,Books,Autonomous University of Nuevo Leon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4102,Gutenberg Bible,NaN,"Johann Gutenberg was born in Mainz, Germany, a...",Biblia,https://www.wdl.org/en/item/4102,https://hdl.loc.gov/loc.wdl/wdl.4102,1.0,Books,Bavarian State Library,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14711,Prayer Book of Emperor Maximilian I,Illuminated Manuscripts from Europe,There are six known copies of this prayer book...,NaN,https://www.wdl.org/en/item/14711,https://hdl.loc.gov/loc.wdl/wdl.14711,1.0,Books,Bavarian State Library,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18175,Dance of Death,NaN,The Middle Rhenish Totentanz (Dance of death) ...,"Der doten dantz mit figuren, clage vnd antwort...",https://www.wdl.org/en/item/18175,https://hdl.loc.gov/loc.wdl/wdl.18175,1.0,Books,Bavarian State Library,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Assuming your DataFrame is named 'df'
row_count = len(loc_df)

print("Number of records in the DataFrame:", row_count)

Number of records in the DataFrame: 1183


In [12]:
# Specify the columns to keep
keep_columns = ['wdl_id', 'title', 'collection', 'description','wdl_url', 'Type of Item', 'Institution','Date Created', 'Dewey Decimal Code', 'Creator', 'Publisher', 'Place of Publication']

Column headers in the DataFrame: ['wdl_id', 'title', 'collection', 'description', 'title_in_original_language', 'wdl_url', 'handle', 'Random', 'Type of Item', 'Institution', 'Photographer', 'Date Created', 'Subject Date', 'Place', 'Dewey Decimal Code', 'Additional Subjects', 'Physical Description', 'Creator', 'Publisher', 'Place of Publication', 'Language', 'Artist', 'Cartographer', 'Note', 'Delineator', 'Author', 'Surveyor', 'Compiler', 'Translator', 'Contributor', 'Engraver', 'Cinematographer', 'Director', 'Illustrator', 'Sponsor', 'Draftsman', 'Editor', 'Corrector', 'Responsible Party', 'Calligrapher', 'Interviewee', 'Interviewer', 'Associated Name', 'Printer', 'Lithographer', 'Donor', 'Recording Engineer', 'Musician', 'Composer', 'Architect', 'References', 'Attributed Name', 'Dedicatee', 'Signer', 'Copyright Holder', 'Collector', 'Printmaker', 'Copyright claimant', 'Performer', 'Owner', 'Commentator', 'Etcher', 'Wood-engraver', 'Scribe', 'Author of Introduction, etc.', 'Author of A

In [21]:
#Isolate the Type of Item for the Item_Type Table
#Calculate the number of unique authors in the DataFrame
item_count = len(loc_df['Type of Item'].unique())
item_type = (loc_df['Type of Item'].unique())
print(item_type)

['Books' 'Journals' 'Manuscripts' 'Maps' 'Newspapers'
 'Prints, Photographs' 'Motion Pictures' 'Sound Recordings']


In [22]:
#Isolate the Collection Type for the Collection_Type Table
#Calculate the number of unique authors in the DataFrame
collection_count = len(loc_df['collection'].unique())
collection_type = (loc_df['collection'].unique())
print(collection_count)
print(collection_type)

139
[nan 'Illuminated Manuscripts from Europe'
 'Arabic and Islamic Science and Its Influence on the Western Scientific Tradition: Mathematics'
 "Saint John's Rare Books" 'Arca Artium Rare Books'
 'United States History' 'Chinese Rare Book Collection'
 'Pre-1801 Imprint Collection' 'African Section Pamphlet Collection'
 'Japanese Rare Book Collection' 'Peace Handbooks'
 'Lessing J. Rosenwald Collection' 'World War I'
 'The Jeannette Expedition of 1879\x9681'
 'Accounts of Whaling Expeditions' 'Joseph Meredith Toner Collection'
 'Brockhaus and Efron Encyclopedic Dictionary'
 'Chinese Books, Manuscripts, Maps, and Prints' 'World History'
 'Galilean Collection' 'Peruvian First Editions'
 'Annual Reports of the Russian-American Company'
 'Arabic and Islamic Science and Its Influence on the Western Scientific Tradition: Technology'
 'Arabic and Islamic Science and Its Influence on the Western Scientific Tradition: Astronomy'
 'Robert Burns Chapbook Collection' 'The Kharkiv Democritus'
 'Cro

In [23]:
#Isolate the Collection Type for the Collection_Type Table
#Calculate the number of unique authors in the DataFrame
Institution_count = len(loc_df['Institution'].unique())
Institution_type = (loc_df['Institution'].unique())
print(Institution_count)
print(Institution_type)

100
['Austrian National Library' 'Autonomous University of Nuevo Leon'
 'Bavarian State Library' 'Bibliotheca Alexandrina'
 'Boris Yeltsin Presidential Library'
 'Central Library of the Bulgarian Academy of Sciences'
 'Complutense University of Madrid' 'Folger Shakespeare Library'
 'Hill Museum & Manuscript Library' 'John Carter Brown Library'
 'Library of Congress'
 'National Academic Library of the Republic of Kazakhstan, Astana'
 'National Central Library' 'National Central Library of Florence'
 'National Diet Library' 'National Library and Archives of Quebec'
 'National Library of Brazil' 'National Library of China'
 'National Library of Colombia' 'National Library of France'
 'National Library of Korea' 'National Library of Peru'
 'National Library of Romania' 'National Library of Russia'
 'National Library of Sweden' 'National Library of Uganda'
 'National Library of Viet Nam' 'Qatar National Library'
 'Romanian Academy Library' 'Russian State Library'
 'Slovak National Library' 

In [24]:
#Isolate the Collection Type for the Collection_Type Table
#Calculate the number of unique authors in the DataFrame
Publisher_count = len(loc_df['Publisher'].unique())
Publisher_type = (loc_df['Publisher'].unique())
print(Publisher_count)
print(Publisher_type)

270
['Stephanus Creuzer' 'Guillermo Glindon' nan 'Johann Schönsperger'
 'Heinrich Knoblochtzer' 'Johann Gutenberg' 'Aldine Press'
 'Imprimerie impériale' 'Saada Publishing'
 'Council of the Ottoman Encyclopedia Press'
 'Russian Imperial Academy of Science' 'A. Damianova'
 'Knigopechatnitsa (tipografiia) na ?osifa Kopa?niga'
 'Kniazhesko-sr?bska (Pravitelstvena) knigopechatnia'
 'Pechatnitsa (shtamparnitsa) na Dim. Panichkov'
 'Royal Serbian Publishing House' 'Royal Serbian Typographers'
 'Jacobo Cromberger' 'Typographia Medicea' 'Johann Fust' 'Stephen Daye'
 "C. Scribner's Sons" 'Wuying Hall' 'Printed for John Stockdale'
 'E.S. Mittler und Sohn' 'Imprimerie Administrative' 'Thomas Hamilton'
 'Rodwell and Martin' 'Armand Colin et cie' 'Ferdinand Hirt & Sohn'
 'G. and J. Robinson' 'Kirk and Mercein' 'Imprimerie Jean Cussac'
 'John Murray' 'Publicity Promotions' 'Harper & Brothers' 'Edward Arnold'
 'Yao Kanb?' 'Lauffer und Stolp' 'Wu Aheng'
 'Macmillan and Company, Limited' 'H.M. Statione